In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

from sklearn.model_selection import train_test_split 
import lightgbm as lgb

import cufflinks as cf
cf.go_offline()

import xgboost as xgb 
from lightgbm import LGBMClassifier


In [ ]:
#Load the tran and test dataframes 
application_train=
application_test=

In [ ]:
#Features dataframe using train and test set 
def featuresDF(df_train,df_test,target):
    training=df_train
    testing=df_test

    training['is_train']=1
    training['is_test']=0
    testing['is_train']=0
    testing['is_test']=1
    
    train_X=training.drop([target],axis=1)
    test_X=testing
    data=pd.concat([train_X,test_X],axis=0)
    return data
data=featuresDF(application_train,application_test,"Comment")

In [ ]:
#Target variable extraction 
def target(df_train,target):
    training=df_train
    Y=training[target]
    return Y
    
Y=target(application_train,'TARGET')

In [ ]:
#Missing values Analysis 
def missingValuesAnalysis(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df=missingValuesAnalysis(application_train)

In [ ]:
#Function to capture numerical variables 
def _get_numerical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype!='object']
    return feats 
numFeatures=_get_numerical_features(data)

In [ ]:
#Function to capture categorical variables 
def _get_categorical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype=='object']
    return feats 
cats=_get_categorical_features(data)

In [ ]:
#Function to factorize categorical variables 
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 
#data=_factorize_categoricals(data,cats)

In [ ]:
#Function to convert categorical variables into dummy variables 
def _get_dummies(df,cats):
    for col in cats:
         df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df
#data=_get_dummies(data,cats)

In [ ]:
#Doing factorize or dummy variable encoding 
def encoding(df,flag):
    data_cats = _get_categorical_features(df)
    if(flag==1):
        df = _factorize_categoricals(df,data_cats)
    else:
        df=_get_dummies(df,data_cats)
        df=df.drop(data_cats,axis=1)
    return df
data=encoding(data,0)

In [ ]:
ignoreFeatures = ['CUST_ID']
importantFeatures = [col for col in data.columns if col not in ignoreFeatures]

In [ ]:
#Splitting into train and test set 
ignoreFeatures = ['CUST_ID', 'is_train', 'is_test']
importantFeatures = [col for col in data.columns if col not in ignoreFeatures]
trainX = data[data['is_train'] == 1][importantFeatures]
testX = data[data['is_test'] == 1][importantFeatures]
x_train, x_val, y_train, y_val = train_test_split(trainX, Y, test_size=0.2, random_state=18)

# Method 1 with train and test split - lgb.train 

In [ ]:
#Method1---Fitting LGB model with lgb.train 
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_eval = lgb.Dataset(data=x_val, label=y_val)
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'max_bin':200,
          'learning_rate': 0.01, 'num_leaves': 150, 'num_iteration':5000, 'verbose': 0 ,
          'colsample_bytree':.9, 'subsample':.9, 'max_depth':30, 'reg_alpha':1, 'reg_lambda':1, 
          'min_split_gain':.01, 'min_child_weight':4}

model = lgb.train(params,lgb_train,valid_sets=lgb_eval,early_stopping_rounds=150,verbose_eval=200)

In [ ]:
#Prediction 
preds = model.predict(testX)
test_id=application_test['CUST_ID']
sub_lgb = pd.DataFrame()
sub_lgb['CUST_ID'] = test_id
sub_lgb['COMMENT'] = preds
sub_lgb.to_csv("Prediction.csv", index=False)
sub_lgb.head()

# Method 2 with train and test split - lgbmclassifier 

In [ ]:
#Method2----Fitting LGB model with lgb classifier 
model = LGBMClassifier(boosting_type='gbdt', objective='multiclass',
                       num_class=9,early_stopping = 50,num_iteration=10000,num_leaves=31,
                       is_enable_sparse='true',tree_learner='data',min_data_in_leaf=600,max_depth=4,
                       learning_rate=0.01, n_estimators=675, max_bin=255, subsample_for_bin=50000, 
                       min_split_gain=5, min_child_weight=5, min_child_samples=10, subsample=0.995, 
                       subsample_freq=1, colsample_bytree=1, reg_alpha=0, 
                       reg_lambda=0, seed=0, nthread=-1, silent=True)

#Fit to training data
eval_set = [(x_train, y_train), (x_val, y_val)]
model=model.fit(x_train, y_train, eval_metric=["error","auc"], eval_set=eval_set, verbose=True)
#Generate Predictions
y_pred=model.predict_proba(test_x)


In [ ]:
#Variable importance 
lgb.plot_importance(model, figsize=(12, 25), max_num_features=100)

#  Method 3 with cross validation -No train test split --lgb.cv

In [ ]:
#Fitting LGB model using Cross validation(CV method) (lgb.cv)
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'max_bin':200,
          'learning_rate': 0.01, 'num_leaves': 150, 'num_iteration':5000, 'verbose': 0 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':9, 'reg_alpha':1, 'reg_lambda':1, 
          'min_split_gain':.01, 'min_child_weight':1}
lgb_train = lgb.Dataset(data=trannX, label=Y)
cvresult = lgb.cv(params, lgb_train, num_boost_round = 1000, nfold = 5, metrics = 'auc', early_stopping_rounds = 10, seed = 10)

# LightGBM model with tunning -gridserarch 

In [ ]:
from sklearn.grid_search import GridSearchCV
# using features, labels, cv_folds, model from previous example
#model.set_params(n_estimators = 1600)
param_test1 = {
'max_depth': [6,7,8,9],
'min_child_weight': [1,2,3,4],
}
features=trainX
labels=Y
gsearch1 = GridSearchCV(estimator = model, param_grid = param_test1, scoring = 'roc_auc', n_jobs = 4, iid = False,verbose = 2)
gsearch1.fit(features,labels)
#print gsearch1.grid_scores_
print(gsearch1.best_params_)
print(gsearch1.best_score_)